<div class="alert alert-block alert-warning">
<b>Warning:</b>
 To successfuly run these analyses you should run the following notebooks before:
</div>

- [preprocessing](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/preprocessing_data.ipynb)

- [formating and annotating](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/formatting_and_annotating.ipynb)

- [signature extraction](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/signature_extraction.ipynb)

- [regression](http://nbviewer.jupyter.org/urls/bitbucket.org/bbglab/mutfootprints/raw/master/regression.ipynb)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import gzip
import pickle
import json
from itertools import combinations, product

import scipy
from scipy.spatial.distance import cosine
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from sklearn import metrics

import matplotlib.pyplot as plt
from matplotlib import rc

In [ ]:
def plotParam():
    """
    plot parameters
    :return:
    """
    plt.rcParams['font.sans-serif'] = ['arial']
    plt.rcParams['font.size'] = 14
    plt.rcParams['font.family'] = ['sans-serif']
    plt.rcParams['svg.fonttype'] = 'none'
    plt.rcParams['mathtext.fontset'] = 'custom'
    plt.rcParams['mathtext.cal'] = 'arial'
    plt.rcParams['mathtext.rm'] = 'arial'

plotParam()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#### The following two files loaded as signatures_SA and signatures_SP were retrieved from [Synapse](https://www.synapse.org):

"**SignatureAnalyzer_COMPOSITE_SBS_W96"** downloaded from [syn11738310](https://www.synapse.org/#!Synapse:syn11738310)
"**sigProfiler_SBS_signatures**" downloaded from [syn11738319](https://www.synapse.org/#!Synapse:syn11738319)

In [ ]:
signatures_SA = pd.read_csv('data/signatures_files/PCAWG/SignatureAnalyzer_COMPOSITE_SBS_W96.signature.031918.txt', sep='\t')
signatures_SP = pd.read_csv('data/signatures_files/PCAWG/sigProfiler_SBS_signatures_2018_03_28.indexed.csv', sep=',')

# ChemoMut results

In [ ]:
extraction_path = 'data/hartwig/signatures/extraction/results/'
sp_proc_path = os.path.join(extraction_path, 'SigProfiler/snvs/processes/PanNoSkinNoUnknown.snvs/PanNoSkinNoUnknown.snvs.processes.tsv')
sa_proc_path = os.path.join(extraction_path, 'SignatureAnalyzer/snvs/processes/Pan_full/Pan_full.processes.tsv')
sp_exp_path = os.path.join(extraction_path, 'SigProfiler/snvs/exposures/PanNoSkinNoUnknown.snvs/PanNoSkinNoUnknown.snvs.exposures.tsv')
sa_exp_path = os.path.join(extraction_path, 'SignatureAnalyzer/snvs/exposures/Pan_full/Pan_full.exposures.tsv')
sp_proc_dbs_path = 'data/hartwig/signatures/extraction/results/SigProfiler/dbs/processes/PanNoSkinNoUnknown.dbs/PanNoSkinNoUnknown.dbs.processes.tsv'
sa_proc_dbs_path = os.path.join(extraction_path, 'SignatureAnalyzer/dbs/processes/Pan/Pan.processes.tsv')

In [ ]:
# load processes

sp_proc = pd.read_csv(sp_proc_path, sep='\t', index_col=0)
sa_proc = pd.read_csv(sa_proc_path, sep='\t', index_col=0)

sp_exp = pd.read_csv(sp_exp_path, sep='\t', index_col=0)
sa_exp = pd.read_csv(sa_exp_path, sep='\t', index_col=0)

sig_dict = {
    'proc': {'sp': sp_proc, 'sa': sa_proc}, 
    'exp': {'sp': sp_exp, 'sa': sa_exp}
}

sp_proc_dbs = pd.read_csv(sp_proc_dbs_path, sep='\t', index_col=0)
sa_proc_dbs = pd.read_csv(sa_proc_dbs_path, sep='\t', index_col=0)

sig_dict_dbs = {'proc': {'sp': sp_proc_dbs, 'sa': sa_proc_dbs}}

In [ ]:
columns_sa = set(sig_dict['exp']['sa'].columns)
columns_sp = set(sig_dict['exp']['sp'].columns)
samples = columns_sa.intersection(columns_sp)

Sa = sig_dict['exp']['sa'][samples].values.T
Sp = sig_dict['exp']['sp'][samples].values.T

index_dict_sa = {v: i for i, v in enumerate(sig_dict['exp']['sa'].index)}
index_dict_sp = {v: i for i, v in enumerate(sig_dict['exp']['sp'].index)}

## Optimization problem

Given two signatures $C_1$ and $C_2$ and a target signature $S$, we want to assert whether there is a good convex combination $C(a) = a C_1 + (1-a) C_2$ that approximates well $S$. The following natural constraint must hold: $a \geq 0$ (positivity). We use the cosine distance $\cos(C(a), S)$ as approximation criterion.

In [ ]:
def cosine_objective(c, target):
    """c: array-like with shape: n_processes x n_channels"""
    
    def objective(w):
        
        comb = np.dot(w, c)
        dot_prod = np.dot(comb, target)
        cosine = dot_prod / (np.linalg.norm(comb) * np.linalg.norm(target))
        return 1 - cosine
    
    return objective
    

def weight_optimize(c, target):

    n_processes, n_channels = c.shape
    obj_func = cosine_objective(c, target)
    bnds = [(0, 1) for _ in range(n_processes)]
    cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})
    res = minimize(obj_func, 1/n_processes * np.ones(n_processes), method='SLSQP', bounds=bnds, constraints=cons)
    cosine_similarity = 1 - obj_func(res.x)
    return res.x, cosine_similarity

### a little testing...

In [ ]:
# small test

c1 = signatures_SP['SBS1'].values
c2 = signatures_SP['SBS3'].values
c3 = signatures_SP['SBS5'].values
a1, a2, a3 = 0.5, 0.3, 0.2  # set weights for target convex combination
convex = a1 * c1 + a2 * c2 + a3 * c3
c = np.array([c1, c2, c3])

In [ ]:
(ahat1, ahat2, ahat3), cosine = weight_optimize(c, convex)

In [ ]:
ahat1, ahat2, ahat3

In [ ]:
cosine

## Utils

In [ ]:
def closest_combination(s, n_processes, method='sa'):
    """
    Args
        s: mutational profile
        n_processes: number of processes allowed to find the best approximation to s
    """
    
    pool = np.array(list(sig_dict['proc'][method].columns))
    cosine = 0
    c = np.empty((n_processes, len(s)))
    all_results = []
    
    print('progress print-out...\n')
    for indices in combinations(range(len(pool)), n_processes):
        indices = list(indices)
        w, cos = weight_optimize(sig_dict['proc'][method].iloc[:, indices].values.T, s)
        cos = np.round(cos, 3)
        w = tuple(map(lambda x: np.round(x, 3), w))
        all_results.append((list(pool[indices]), w, cos))
        if cos > cosine:
            cosine = cos
            c = np.array(pool[indices])
            weights = w
            print(cosine, list(c), weights)  # progress print-out: cosine similarity, 
                                             # candidate components and convex weights

    all_results = sorted(all_results, key=lambda x: x[2], reverse=True)
    return list(c), weights, cosine, all_results

def single_lsq(A, b):
    
    lb = np.zeros(A.shape[1])  # lower bound
    ub = lb + 1    
    res = scipy.optimize.lsq_linear(A, b, bounds=(lb, ub))
    return res.x / np.sum(res.x)

def deconstruct_splits(sp_signature, columns=None):
    
    if columns is None:
        indexes = list(range(Sa.shape[1]))
    else:
        indexes = [index_dict_sa[c] for c in columns]
    j = index_dict_sp[sp_signature]
    x = single_lsq(Sa[:, indexes], Sp[:, j])
    return x

### a little testing...

In [ ]:
weights = deconstruct_splits('1_SBS31_0.968153_0.98', columns=['14_1', '21_SBS31_0.953955_1'])

In [ ]:
weights[0], weights[1]

## What are the combinations of two sa-signatures that best explain a given sp-signature?

In [ ]:
# what are the available SP targets?

sig_dict['proc']['sp'].keys().tolist()

In [ ]:
# SigProfiler SBS31

target_label = '1_SBS31_0.968153_0.98'
sig = sig_dict['proc']['sp'][target_label].values
c, weights, cosine, all_results = closest_combination(sig, 2, method='sa')

In [ ]:
# SigProfiler SBS20

target_label = '20_0.92'
sig = sig_dict['proc']['sp'][target_label].values
c, weights, cosine, all_results = closest_combination(sig, 2, method='sa')

### plot utils

In [ ]:
# stacked barplot representation

def stacked_bar_plot(s1_weights, s2_weights, other_weights, xlabels, title):

    bars1 = np.add(s1_weights, s2_weights).tolist()
    r = list(range(len(xlabels)))
    names = xlabels
    barWidth = 0.3

    fig, ax = plt.subplots(figsize=(3, 5))
    
    ax.bar(r, s1_weights, color='green', edgecolor='white', width=barWidth, alpha=0.6)
    ax.bar(r, s2_weights, bottom=s1_weights, color='purple', alpha=0.8, edgecolor='white', width=barWidth)
    ax.bar(r, other_weights, bottom=bars1, color='grey', edgecolor='white', width=barWidth)
    
    # remove spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    # custom x axis
    ax.set_xticks(r)
    ax.set_xticklabels(names)
    ax.set_xlabel("cosine similarity")
    rc('font', weight='bold')
    
    plt.title(title)

### Explanation of E-SBS20

In [ ]:
title = 'E-SBS20'
xlabels = ['0.85']
s14 = [0.433]
s37 = [0.567]
other = [0]

In [ ]:
stacked_bar_plot(s14, s37, other, xlabels, title)

### Explanation of E-SBS31

In [ ]:
title = 'E-SBS31'
xlabels = ['0.97']
s14 = [0.295]
s21 = [0.705]
other = [0]

In [ ]:
stacked_bar_plot(s14, s21, other, xlabels, title)

## doublets

In [ ]:
res_sigprofiler_dbs = pd.read_csv(sp_proc_dbs_path, sep='\t', index_col=0)
res_siganalyzer_dbs = pd.read_csv(sa_proc_dbs_path, sep='\t', index_col=0)
sig_dbs_dict = {'sp': res_sigprofiler_dbs, 'sa': res_siganalyzer_dbs}

In [ ]:
def closest_dbs_combination(s, method='sa'):
    
    pool = sig_dbs_dict[method].columns
    cosine = 0
    c1, c2 = None, None
    target = None
    all_results = []
    for i, j in product(range(len(pool)), repeat=2):
        if i < j:
            c = np.array([sig_dbs_dict[method].iloc[:,i].values, sig_dbs_dict[method].iloc[:,j].values])
            w, cos = weight_optimize(c, s)
            cos = np.round(cos, 3)
            w = tuple(map(lambda x: np.round(x, 3), w))
            all_results.append((pool[i], pool[j], w, cos))
            if cos > cosine:
                cosine = cos
                c1, c2 = pool[i], pool[j]
                weights = w
    all_results = sorted(all_results, key=lambda x: x[3], reverse=True)
    return c1, c2, weights, cosine, all_results

In [ ]:
target_label = '5_DBS5_0.944431_1.0'
sig = sig_dbs_dict['sp'][target_label].values
c1, c2, weights, cosine, all_results = closest_dbs_combination(sig, method='sa')

In [ ]:
c1, c2, weights, cosine

### Explanation of DBS5

In [ ]:
title = 'E-DBS5'
xlabels = ['0.99']
s3 = [0.358]
s9 = [0.642]
other = [0]

In [ ]:
stacked_bar_plot(s3, s9, other, xlabels, title)